GPU check

In [2]:
!nvidia-smi

Wed Aug 27 18:43:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.19                 Driver Version: 561.19         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   42C    P3             12W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, default_data_collator
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import DataCollatorForLanguageModeling

C:\Users\aravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model_path = "gemma_model" 
tokenizer_path = "gemma_tokenizer"  
dataset_path = "A:/gemma270-rag/data/100_sample_query_finetuning.csv"
output_dir = "outputs"

In [5]:
q_lora = False  # True -> QLoRA, False -> LoRA
lora_r, lora_alpha, lora_dropout = 8, 16, 0.05
brevity_B, brevity_lambda = 10, 0.1


In [ ]:
from datasets import load_dataset

dataset_path = "A:/gemma270-rag/data/100_sample_query_finetuning.csv"

ds = load_dataset("csv", data_files=dataset_path)

dataset = ds["train"]

dataset = dataset.remove_columns([col for col in dataset.column_names if col not in ["query", "answer", "theme"]])


def build_prompt(ex):
    prompt = f"Query: {ex['query']}\nAnswer:"
    full = f"Query: {ex['query']}\nAnswer: {ex['answer']}"
    return prompt, full
def preprocess(ex):
    prompt, full = build_prompt(ex)
    
    prompt_ids = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
    )["input_ids"]
    
    full_enc = tokenizer(
        full,
        truncation=True,
        padding="max_length",   
        max_length=512,
    )

    labels = full_enc["input_ids"].copy()
    
    for i in range(len(prompt_ids)):
        if i < len(labels):
            labels[i] = -100
    
    full_enc["labels"] = labels
    return full_enc


In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

In [8]:
if q_lora:
    from transformers import BitsAndBytesConfig
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=bnb_config, device_map="auto")
    model = prepare_model_for_kbit_training(model)
else:
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")

In [ ]:
model.resize_token_embeddings(len(tokenizer))
tokenized_ds = ds["train"].map(
    preprocess,
    remove_columns=ds["train"].column_names
)

target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"]
config = LoraConfig(r=lora_r, lora_alpha=lora_alpha, target_modules=target_modules, lora_dropout=lora_dropout, task_type="CAUSAL_LM")
model = get_peft_model(model, config)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False   
)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # <-- add **kwargs
        labels = inputs.get("labels")
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

#bf16=True, while training in real gpu
args = TrainingArguments(output_dir=output_dir, num_train_epochs=3, per_device_train_batch_size=8, gradient_accumulation_steps=2, learning_rate=2e-4, bf16=True, save_strategy="epoch", logging_steps=50, remove_unused_columns=False)

trainer = CustomTrainer(model=model, args=args, train_dataset=tokenized_ds, tokenizer=tokenizer, data_collator=data_collator)

trainer.train()
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Map: 100%|██████████| 100/100 [00:00<00:00, 621.68 examples/s]
C:\Users\aravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\aravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
C:\Users\aravi\AppData\Local\Temp\ipykernel_24492\3479529581.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(m

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
